<a href="https://colab.research.google.com/github/chiruvishal/MajorProject/blob/main/Trainingroberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random

df = pd.read_csv("dataset.csv")


# Create a new DataFrame with columns 'text' and 'label' (1 for human, 0 for ChatGPT)
human_reviews = df['reviews'].tolist()
chatgpt_reviews = df['generated_reviews'].tolist()
labels = [1] * len(human_reviews) + [0] * len(chatgpt_reviews)
data = {'text': human_reviews + chatgpt_reviews, 'label': labels}
new_df = pd.DataFrame(data)

# Shuffle the DataFrame to randomize the order
new_df = new_df.sample(frac=1).reset_index(drop=True)

# Save the new DataFrame as a CSV file for model training

# Save the dataset with labels


In [ ]:
new_df.head(200)


,text,label
0,This product is incredible! It works perfectly...,0
1,I bought this phone for my daughter and she sa...,0
2,I love my phone! I have taken it all over the ...,1
3,The mobile is Ok but in the description said t...,1
4,\nI had to return because it was not compatibl...,0
...,...,...
195,I was very disappointed with this purchase. Th...,0
196,The phone looks great but it get hot when it u...,1
197,"It was a good product, but the accessories wer...",0
198,This is a real battery of 4000 miliamps. If yo...,1


In [ ]:
new_df.to_csv("labeled-dataset.csv", index=False)


In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00


In [ ]:
# Load the labeled dataset
df = pd.read_csv("labeled-dataset.csv")


In [ ]:
# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Define the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]["text"]
        label = int(self.data.iloc[index]["label"])
        encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, padding="max_length", return_tensors="pt", return_attention_mask=True)
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": label
        }

# Tokenization and DataLoader
max_length = 128  # Adjust as needed
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:
# Initialize the RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 3  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss / len(train_loader)}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Training Loss: 0.4396620714797242
Epoch 2/3, Training Loss: 0.2998114309577954
Epoch 3/3, Training Loss: 0.24418052547786495


In [ ]:
model.eval()
val_loss = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        val_loss += loss.item()
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds)

accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=["ChatGPT", "Human"])
print(f"Validation Loss: {val_loss / len(val_loader)}")
print(f"Accuracy: {accuracy}")
print(report)


Validation Loss: 0.3650563655822885
Accuracy: 0.8599752168525403
              precision    recall  f1-score   support

     ChatGPT       0.85      0.89      0.87       420
       Human       0.87      0.83      0.85       387

    accuracy                           0.86       807
   macro avg       0.86      0.86      0.86       807
weighted avg       0.86      0.86      0.86       807



In [ ]:
model.save_pretrained("fine_tuned_roberta")


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np

# Load the fine-tuned model
model = RobertaForSequenceClassification.from_pretrained("fine_tuned_roberta")

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Define the input review
input_review = "Got this on time it was new worked perfectly fine. I would so recommend this to whoever would like it.(Update) I had the phone less than a year and i had some issues. I did get the phone flashed so it could work under another network. After having it for a few months people would call me and i would pick up i could hear them, but they were not able to hear me. So I took it to and iphone repair shop over three times, the replaced the microphone and it still didn't work. I gave up and switch to another phone. Money down the drain on this one."

# Tokenize the input
input_ids = tokenizer(input_review, return_tensors="pt", padding=True, truncation=True)
input_ids = input_ids["input_ids"]

# Pass the input through the model
with torch.no_grad():
    output = model(input_ids)

# Get the predicted label
predicted_label = int(np.argmax(output.logits))

# Interpret the label (0 for ChatGPT, 1 for human)
if predicted_label == 0:
    print("ChatGPT")
else:
    print("Human")


Human


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np

# Load the fine-tuned model
model = RobertaForSequenceClassification.from_pretrained("fine_tuned_roberta")

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Define the input review
input_review = "The product was good, but the accessories were not of good quality. The charger was not of good quality, and the phone had some sound issues."

# Tokenize the input
input_ids = tokenizer(input_review, return_tensors="pt", padding=True, truncation=True)
input_ids = input_ids["input_ids"]

# Pass the input through the model
with torch.no_grad():
    output = model(input_ids)

# Get the predicted label
predicted_label = int(np.argmax(output.logits))

# Interpret the label (0 for ChatGPT, 1 for human)
if predicted_label == 0:
    print("ChatGPT")
else:
    print("Human")

ChatGPT


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Save the model to the specified path
model.save_pretrained('/content/gdrive/MyDrive/fine_tuned_roberta')
